In [10]:
# ----------------------------------------------
# Bereinigung und Standardisierung pro Jahr (2018–2021)
# ----------------------------------------------

import pandas as pd
from io import StringIO
import matplotlib.pyplot as plt
import os

# Ausgabeordner für Info-Bilder und bereinigte Daten
os.makedirs("output/info", exist_ok=True)

# Dateipfade
files = {
    2018: "AfSBBB_BE_LOR_Strasse_Strassenverkehrsunfaelle_2018_Datensatz.csv",
    2019: "AfSBBB_BE_LOR_Strasse_Strassenverkehrsunfaelle_2019_Datensatz.csv",
    2020: "AfSBBB_BE_LOR_Strasse_Strassenverkehrsunfaelle_2020_Datensatz.csv",
    2021: "AfSBBB_BE_LOR_Strasse_Strassenverkehrsunfaelle_2021_Datensatz.csv"
}

# Einlesen
data = {}
for year, path in files.items():
    df = pd.read_csv(path, sep=';', encoding='latin1', on_bad_lines='skip')
    data[year] = df

# Spaltenharmonisierung 2018: STRZUSTAND + IstSonstig umbenennen
if 'STRZUSTAND' in data[2018].columns:
    data[2018] = data[2018].rename(columns={'STRZUSTAND': 'USTRZUSTAND'})
if 'IstSonstig' in data[2018].columns:
    data[2018] = data[2018].rename(columns={'IstSonstig': 'IstSonstige'})

# LOR_ab_2021 Mapping sicher erzeugen
mapping_frames = []
for y in [2018, 2020, 2021]:
    df = data[y]
    if 'LOR' in df.columns and 'LOR_ab_2021' in df.columns:
        mapping_frames.append(df[['LOR', 'LOR_ab_2021']].dropna())
lor_mapping_df = pd.concat(mapping_frames, ignore_index=True).drop_duplicates(subset='LOR')
lor_mapping = pd.Series(lor_mapping_df['LOR_ab_2021'].values, index=lor_mapping_df['LOR']).to_dict()

# Mapping für 2019 anwenden, falls LOR_ab_2021 fehlt
if 'LOR_ab_2021' not in data[2019].columns or data[2019]['LOR_ab_2021'].isnull().all():
    data[2019]['LOR_ab_2021'] = data[2019]['LOR'].map(lor_mapping)

# Einheitliche Spaltenreihenfolge definieren
columns_order = [
    'OBJECTID', 'LAND', 'BEZ', 'LOR', 'STRASSE', 'LOR_ab_2021', 'UJAHR',
    'UMONAT', 'USTUNDE', 'UWOCHENTAG', 'UKATEGORIE', 'UART', 'UTYP1',
    'ULICHTVERH', 'IstRad', 'IstPKW', 'IstFuss', 'IstKrad', 'IstGkfz',
    'IstSonstige', 'USTRZUSTAND', 'LINREFX', 'LINREFY', 'XGCSWGS84', 'YGCSWGS84']

# Bereinigte Daten pro Jahr erstellen
required_columns = [
    'BEZ', 'LOR_ab_2021', 'UJAHR', 'UMONAT', 'USTUNDE', 'UWOCHENTAG',
    'UKATEGORIE', 'UART', 'UTYP1', 'ULICHTVERH', 'IstRad', 'IstPKW',
    'IstFuss', 'IstKrad', 'IstGkfz', 'IstSonstige', 'USTRZUSTAND',
    'XGCSWGS84', 'YGCSWGS84']

numerical_columns = ['LOR_ab_2021', 'IstSonstige', 'USTRZUSTAND']
valid_ranges = {
    'BEZ': (1, 12),
    'UJAHR': (2000, 2024),
    'UMONAT': (1, 12),
    'USTUNDE': (0, 23),
    'UWOCHENTAG': (1, 7),
    'UKATEGORIE': [1, 2, 3],
    'UART': (0, 9),
    'UTYP1': (1, 7),
    'ULICHTVERH': [0, 1, 2],
    'IstRad': [0, 1], 'IstPKW': [0, 1], 'IstFuss': [0, 1], 'IstKrad': [0, 1], 'IstGkfz': [0, 1], 'IstSonstige': [0, 1],
    'USTRZUSTAND': [0, 1, 2]
}

bereinigte_alle = []

for year in data:
    df = data[year].copy()
    df = df.reindex(columns=columns_order)
    df = df.loc[:, required_columns].dropna()
    df = df.drop_duplicates()
    df[numerical_columns] = df[numerical_columns].apply(pd.to_numeric, errors='coerce')
    df = df.dropna(subset=numerical_columns)
    df[numerical_columns] = df[numerical_columns].astype('int64')
    df['XGCSWGS84'] = df['XGCSWGS84'].astype(str).str.replace(',', '.').astype(float)
    df['YGCSWGS84'] = df['YGCSWGS84'].astype(str).str.replace(',', '.').astype(float)

    for col, rng in valid_ranges.items():
        if isinstance(rng, tuple):
            df = df[(df[col] >= rng[0]) & (df[col] <= rng[1])]
        elif isinstance(rng, list):
            df = df[df[col].isin(rng)]

    winter = [11, 12, 1, 2]
    nacht = (df['USTUNDE'] > 18) | (df['USTUNDE'] < 6)
    df = df[~((df['UMONAT'].isin(winter)) & nacht & (df['ULICHTVERH'] == 0))]
    df = df[(df['XGCSWGS84'].between(13, 14)) & (df['YGCSWGS84'].between(52, 53))]
    teilnehmer = ['IstRad', 'IstPKW', 'IstFuss', 'IstKrad', 'IstGkfz', 'IstSonstige']
    df = df[~(df[teilnehmer] == 0).all(axis=1)]
    df = df.reset_index(drop=True)
    bereinigte_alle.append(df)

# Zusammenfügen aller bereinigten Jahresdaten
gesamt = pd.concat(bereinigte_alle, ignore_index=True)
gesamt.to_csv("bereinigte_daten.csv", index=False)

# Ausgabe der .info()-Daten als Bild pro Jahr
for year, df in data.items():
    buffer = StringIO()
    df.info(buf=buffer)
    info_text = buffer.getvalue()
    plt.figure(figsize=(10, 6))
    plt.text(0.01, 0.99, info_text, fontsize=12, va='top', family='monospace')
    plt.axis('off')
    plt.tight_layout()
    plt.savefig(f"output/info/data_{year}_info_output.png", dpi=100, bbox_inches="tight")
    plt.close()

print("Daten pro Jahr bereinigt und zusammengeführt gespeichert.")

Daten pro Jahr bereinigt und zusammengeführt gespeichert.
